Google Doc with screenshots & explaination - https://docs.google.com/document/d/1aHRmN0iSI_cRYjV-hEKrSg3YVGE7fhSNH-XwItpAXO4/edit?usp=sharing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.inspection import permutation_importance

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from IPython.display import display, clear_output
import ipywidgets as widgets

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/sce-energy-data.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head())
print("Columns:", df.columns.tolist())

Shape: (8340, 8)


,Zip\nCode,Month,Year,Customer\nClass,Combined,TotalAccounts,TotalkWh,AveragekWh
0,90002,6.0,2024.0,Residential,N,3255.0,1225332.0,376.446083
1,90003,6.0,2024.0,Residential,N,NaN,NaN,NaN
2,90004,4.0,2024.0,Residential,N,NaN,NaN,NaN
3,90008,6.0,2024.0,Residential,N,674.0,318086.0,471.937685
4,90018,6.0,2024.0,Residential,N,NaN,NaN,NaN


Columns: ['Zip\nCode', 'Month', 'Year', 'Customer\nClass', 'Combined', 'TotalAccounts', 'TotalkWh', 'AveragekWh']


In [6]:
# Clean column names
df = df.copy()
df.columns = [c.replace("\n", "").strip() for c in df.columns]

# Encode target
df = df[df["Combined"].isin(["Y", "N"])].copy()
df["y"] = (df["Combined"] == "Y").astype(int)

print(df.shape)
display(df.head())
print(df.columns.tolist())

(8340, 9)


,ZipCode,Month,Year,CustomerClass,Combined,TotalAccounts,TotalkWh,AveragekWh,y
0,90002,6.0,2024.0,Residential,N,3255.0,1225332.0,376.446083,0
1,90003,6.0,2024.0,Residential,N,NaN,NaN,NaN,0
2,90004,4.0,2024.0,Residential,N,NaN,NaN,NaN,0
3,90008,6.0,2024.0,Residential,N,674.0,318086.0,471.937685,0
4,90018,6.0,2024.0,Residential,N,NaN,NaN,NaN,0


['ZipCode', 'Month', 'Year', 'CustomerClass', 'Combined', 'TotalAccounts', 'TotalkWh', 'AveragekWh', 'y']


In [7]:
# Drop rows missing essential numeric predictors
df = df.dropna(subset=["TotalAccounts", "TotalkWh", "AveragekWh"])

print("After dropping NaNs:", df.shape)

After dropping NaNs: (3122, 9)


In [8]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Combined", "y"])
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)

(2341, 7) (781, 7)


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

cat_cols = ["CustomerClass"]
num_cols = [c for c in X_train.columns if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

print("Numeric:", num_cols)
print("Categorical:", cat_cols)

Numeric: ['ZipCode', 'Month', 'Year', 'TotalAccounts', 'TotalkWh', 'AveragekWh']
Categorical: ['CustomerClass']


In [10]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

MODELS = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    "GB": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(kernel="rbf", probability=True, random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(64,32), max_iter=800, random_state=42),
}

def sens_spec(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sens = tp / (tp + fn)
    spec = tn / (tn + fp)
    return sens, spec

results = []

for name, model in MODELS.items():
    pipe = Pipeline([
        ("prep", preprocess),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    f1 = f1_score(y_test, preds)
    acc = accuracy_score(y_test, preds)
    sens, spec = sens_spec(y_test, preds)

    results.append({
        "Model": name,
        "F1": f1,
        "Accuracy": acc,
        "Sensitivity": sens,
        "Specificity": spec
    })

baseline_results = (
    pd.DataFrame(results)
    .sort_values("F1", ascending=False)
    .reset_index(drop=True)
)

display(baseline_results)

,Model,F1,Accuracy,Sensitivity,Specificity
0,RF,0.700441,0.825864,0.633466,0.916981
1,GB,0.649770,0.805378,0.561753,0.920755
2,MLP,0.544529,0.770807,0.426295,0.933962
3,SVM,0.520000,0.784891,0.362550,0.984906


In [11]:
from sklearn.inspection import permutation_importance

best_model_name = baseline_results.loc[0, "Model"]
best_model = MODELS[best_model_name]

best_pipe = Pipeline([
    ("prep", preprocess),
    ("model", best_model)
])

best_pipe.fit(X_train, y_train)

perm = permutation_importance(
    best_pipe, X_test, y_test,
    n_repeats=10, random_state=42, scoring="f1"
)

importance = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": perm.importances_mean
}).sort_values("Importance", ascending=False)

display(importance)

,Feature,Importance
3,CustomerClass,0.180990
0,ZipCode,0.120401
6,AveragekWh,0.116357
4,TotalAccounts,0.101237
5,TotalkWh,0.040489
2,Year,0.000000
1,Month,-0.046760


In [12]:
def resample_customer_class(X, y, focus_class, target_share):
    """
    Change distribution of CustomerClass in TRAINING data only.
    target_share = desired fraction of focus_class in data.
    """
    df_tmp = X.copy()
    df_tmp["y"] = y.values

    focus = df_tmp[df_tmp["CustomerClass"] == focus_class]
    others = df_tmp[df_tmp["CustomerClass"] != focus_class]

    n_total = len(df_tmp)
    n_focus_target = int(target_share * n_total)
    n_other_target = n_total - n_focus_target

    focus_rs = focus.sample(
        n=n_focus_target,
        replace=(n_focus_target > len(focus)),
        random_state=42
    )
    others_rs = others.sample(
        n=n_other_target,
        replace=(n_other_target > len(others)),
        random_state=42
    )

    df_resampled = (
        pd.concat([focus_rs, others_rs])
        .sample(frac=1, random_state=42)
    )

    y_new = df_resampled["y"]
    X_new = df_resampled.drop(columns=["y"])

    return X_new, y_new

In [13]:
def run_muller_loop(Xtr, ytr, Xte, yte):
    rows = []
    fitted = {}

    for name, model in MODELS.items():
        pipe = Pipeline([
            ("prep", preprocess),
            ("model", model)
        ])
        pipe.fit(Xtr, ytr)
        preds = pipe.predict(Xte)

        f1 = f1_score(yte, preds)
        sens, spec = sens_spec(yte, preds)

        rows.append({
            "Model": name,
            "F1": f1,
            "Sensitivity": sens,
            "Specificity": spec
        })
        fitted[name] = (pipe, preds)

    return (
        pd.DataFrame(rows)
        .sort_values("F1", ascending=False)
        .reset_index(drop=True),
        fitted
    )

In [15]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

focus_classes = sorted(X_train["CustomerClass"].unique())

share_slider = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.1,
    description="Target share",
    continuous_update=False
)

class_dropdown = widgets.Dropdown(
    options=focus_classes,
    value=focus_classes[0],
    description="CustomerClass"
)

model_dropdown = widgets.Dropdown(
    options=list(MODELS.keys()),
    value=baseline_results.loc[0, "Model"],
    description="Confusion model"
)

out = widgets.Output()

def update_dashboard(change=None):
    with out:
        clear_output(wait=True)

        X_rs, y_rs = resample_customer_class(
            X_train, y_train,
            class_dropdown.value,
            share_slider.value
        )

        results, fitted = run_muller_loop(X_rs, y_rs, X_test, y_test)
        display(results)

        pipe, preds = fitted[model_dropdown.value]
        cm = confusion_matrix(y_test, preds)

        fig, ax = plt.subplots(figsize=(4,4))
        ConfusionMatrixDisplay(cm).plot(ax=ax, colorbar=False)
        ax.set_title(
            f"{model_dropdown.value} | "
            f"{class_dropdown.value} share={share_slider.value}"
        )
        plt.show()

        # Sensitivity vs Specificity plot
        fig, ax = plt.subplots(figsize=(5,4))
        ax.scatter(results["Specificity"], results["Sensitivity"])
        for _, r in results.iterrows():
            ax.text(r["Specificity"], r["Sensitivity"], r["Model"])
        ax.set_xlim(0,1)
        ax.set_ylim(0,1)
        ax.set_xlabel("Specificity")
        ax.set_ylabel("Sensitivity")
        ax.set_title("Sensitivity vs Specificity")
        plt.show()

share_slider.observe(update_dashboard, names="value")
class_dropdown.observe(update_dashboard, names="value")
model_dropdown.observe(update_dashboard, names="value")

ui = widgets.VBox([
    class_dropdown,
    share_slider,
    model_dropdown,
    out
])

display(ui)
update_dashboard()

We first trained a baseline Muller loop on the original data distribution. Using permutation importance, CustomerClass was identified as the most influential feature. We then dynamically altered the distribution of CustomerClass using upsampling and downsampling while keeping the test set fixed. As the distribution changed, model performance metrics (F1, sensitivity, specificity) and confusion matrices changed noticeably, demonstrating the sensitivity of models to feature-level data distribution shifts.